## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

## Execício Spark

Realizar análise do arquivo result.csv
O dataset results.csv e um conjunto de dados que inclui resultados de
jogos internacionais de futebol, desde o primeiro jogo oficial em 1972
até 2019. Os jogos variam de Copa do Mundo FIFA à FIFI Wild Cup e
amistosos. Os jogos são estritamente internacionais masculinos e os
dados não incluem Jogos Olímpicos ou jogos em que pelo menos uma das
equipes foi a seleção B do país, sub-23 ou uma seleção da liga.
O dataset foi retirado do Kaggle.

Metadados:

date - data da partida <br>
home_team - time mandante <br>
away_team - time visitante <br>
home_score - placar do time mandante<br>
away_score - placar do time visitante<br>
tournament - torneio<br>
city - cidade onde a partida foi realizada<br>
country - país onde a partida foi realizada<br>
neutral - desconsiderar

Importar o arquivo no Spark e responder as perguntas abaixo utilizando
python ou SQL.
A atividade por ser realizada instalando o Spark na máquina local,
utilizando docker ou o ambiente Databricks Community.
Para cada questão, informar a resposta e o código utilizado para chegar
ao resultado.

1 - quantos registro existem na base?<br>
2 - quantas equipes únicas mandantes existem na base?<br>
3 - quantas vezes as equipes mandantes saíram vitoriosas?<br>
4 - quantas vezes as equipes visitantes saíram vitoriosas?<br>
5 - quantas partidas resultaram em empate?<br>
6 - quantas partidas foram realizadas em cada país?<br>
7 - qual país teve mais partidas?<br>
8 - qual a partida com maior número de gols?<br>
9 - qual a maior goleada?<br>
10 - quantos jogos ocorreram no Brasil?

In [0]:
# File location and type
file_location = "/FileStore/tables/results-2.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Values
#display(df)

# Describe
desc = df.describe()
#display(desc)

# 1 - quantos registro existem na base?
result1 = df.count()
print("1) Quantidade de valores na base = " + str(result1))

# 2 - quantas equipes únicas mandantes existem na base?
result2 = df.select("home_team").distinct().count()
print("2) Quantidade de equipes únicas mandantes = " + str(result2))

# 3 - quantas vezes as equipes mandantes saíram vitoriosas?
result3 = df.where(df.home_score > df.away_score).count()
print("3) Quantidade de vezes que as equipes mandantes saíram vitoriosas = " + str(result3))

# 4 - quantas vezes as equipes visitantes saíram vitoriosas?
result4 = df.where(df.home_score < df.away_score).count()
print("4) Quantidade de vezes que as equipes visitantes saíram vitoriosas = " + str(result4))

# 5 - quantas partidas resultaram em empate?
result5 = df.where(df.home_score == df.away_score).count()
print("5) Quantidade de vezes que as partidas resultaram em empate = " + str(result5))

# 6 - quantas partidas foram realizadas em cada país?
print("6) quantas partidas foram realizadas em cada país =")
df.groupBy("country").count().show();

# 7 - qual país teve mais partidas?
print("7) país teve mais partidas =")
df.groupBy("country").count().orderBy("count",ascending=False).show(n=1);

# 8 - qual a partida com maior número de gols?
print("8) a partida com maior número de gols =")
df.registerTempTable('tab')
spark.sql('select home_team, away_team, home_score, away_score, (home_score + away_score) as gols from tab').orderBy("gols",ascending=False).show(n=1)

# 9 - qual a maior goleada?
print("9) a maior goleada =")
spark.sql('select home_team, away_team, home_score, away_score, CASE WHEN (home_score - away_score) < 0 THEN ((home_score - away_score) * -1) ELSE (home_score - away_score) END as goleada from tab').orderBy("goleada",ascending=False).show(n=1)

# 10 - quantos jogos ocorreram no Brasil?
result10 = df.where(df.country == "Brazil").count()
print("10) quantidade de jogos que ocorreram no Brasil = " + str(result10))


1) Quantidade de valores na base = 40839
2) Quantidade de equipes únicas mandantes = 309
3) Quantidade de vezes que as equipes mandantes saíram vitoriosas = 19856
4) Quantidade de vezes que as equipes visitantes saíram vitoriosas = 11552
5) Quantidade de vezes que as partidas resultaram em empate = 9431
6) quantas partidas foram realizadas em cada país =
+-------------------+-----+
 country|count|
+-------------------+-----+
 Chad| 40|
 Russia| 209|
 Paraguay| 218|
 Anguilla| 12|
 Yemen| 58|
U.S. Virgin Islands| 15|
 Chinese Taipei| 47|
 Senegal| 202|
 Sweden| 637|
 Guyana| 98|
 Philippines| 83|
 Burma| 79|
 Jersey| 68|
 Eritrea| 16|
 Netherlands Guyana| 29|
 Djibouti| 21|
 Tonga| 12|
 Singapore| 368|
 Malaysia| 644|
 Fiji| 139|
+-------------------+-----+
only showing top 20 rows

7) país teve mais partidas =
+-------------+-----+
 country|count|
+-------------+-----+
United States| 1144|
+-------------+-----+
only showing top 1 row

8) a partida com maior número de gols =
+---------+--------------+----------+----------+----+
home_team| away_team|home_score|away_score|gols|
+---------+--------------+----------+----------+----+
Australia|American Samoa| 31| 0|31.0|
+---------+--------------+----------+----------+----+
only showing top 1 row

9) a maior goleada =
+---------+--------------+----------+----------+-------+
home_team| away_team|home_score|away_score|goleada|
+---------+--------------+----------+----------+-------+
Australia|American Samoa| 31| 0| 31.0|
+---------+--------------+----------+----------+-------+
only showing top 1 row

10) quantidade de jogos que ocorreram no Brasil = 529